In [1]:
from packages import Exam
import numpy as np
path = ["./files/result/csv/top_sample.csv",
        "./files/result/csv/back_sample.csv"]

exam = Exam.Exam(path)
exam.get_marks()
exam.get_student()
exam.parts.append(Exam.SingleAlphabetPart(exam.marks[0][4:9, 1:27],["A","Q","B","H","P"],[2]*5))
exam.parts.append(Exam.SingleAlphabetPart(exam.marks[0][10:20, 1:27],["A","E","B","C","H","I","J","M","N","O"],[2]*10))
exam.parts.append(Exam.DualNumberPart(
    marks = np.vstack([exam.marks[0][21:37, 4:27], exam.marks[0][3:7, 31:54]]),
    collects =["A","E","B","C","H","I","J","M","N","O"],
    allocations = [6]*5
    ))
exam.scoring()

print(exam.score)

scoring start of Q 1
scoring start of Q 2
scoring start of Q 3
question scoring error
question scoring error
question scoring error
question scoring error
question scoring error
30


In [ ]:
from packages import Marksheet

marker_positions = [
    ( 2, 26),
    ( 2, 28),
    ( 2, 56),
    ( 3, 24),
    (40,  2),
    (40, 56),
]

sheet = Marksheet(path = "./files/scan/back_sample.jpg", 
                  row = 41, column = 57, 
                  marker_positions = marker_positions,
                  binary_threshold=240,
                  monitor= True)
sheet.read()
sheet.rotation()
sheet.aliment()
sheet.draw_grid()
sheet.threshold_check()
sheet.make_grids()
sheet.mark_check()
sheet.save()

In [ ]:
from packages import Marksheet

marker_positions = [
    ( 2, 28),
    ( 2, 56),
    (40,  2),
    (40,  4),
    (40, 56),
]

sheet = Marksheet(path = "./files/scan/top_sample.jpg", 
                  row = 41, column = 57, 
                  marker_positions=marker_positions,
                  binary_threshold=240,
                  monitor= True)

sheet.read()
sheet.rotation()
sheet.aliment()
sheet.draw_grid()
sheet.threshold_check()
sheet.make_grids()
sheet.mark_check()
sheet.save()


In [ ]:
def extract_significant_digit(value):
    # 浮動小数点型の値を1桁の有効桁数で文字列に変換
    formatted_value = format(value, ".0e")
    # フォーマットされた文字列から数値部分を抽出
    significant_digit = int(formatted_value.split('e')[0])
    return significant_digit

# テスト例
values = [1234.5678, 0.00456, -789.01, 0.0]
significant_digits = [extract_significant_digit(v) for v in values]

print(significant_digits)  # ['1.2', '4.6', '-7.9', '0.0']

In [ ]:
import numpy as np

# 10個の乱数要素の配列を作成
random_array = np.random.rand(10)

# 要素すべての積を計算
product = np.prod(random_array)

print("ランダム配列:", random_array)
print("要素すべての積:", product)